# Mean and std for observational data for VALNA

A separate script to find mean and std for observational data for the following metrics to evaluate North Altantic biases:

   * AMOC at 26.5N at max. depth
   * OHT at 26.5N
   * Mean SST anomaly off Newfoundland
   * Mean SSS anomaly in Labrador Sea
   * GS separation latitude
   * NA current latitude
   * Mixed layer depth in Labrador Sea in March 
   * Subpolar gyre heat content 
   
   * Subpolar gyre strength - COME BACK TO
   * Overflow bottom temperature - COME BACK TO
  


In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import netCDF4 as nc

In [2]:
mdir = '/data/users/smoreton/Obs/'

## AMOC and OHT at RAPID

In [3]:
#2004-2018 annual av
ds = xr.open_dataset(mdir + 'RAPID/mocha_mht_data_ERA5_v2018_2.nc')

In [4]:
df = ds.to_dataframe().reset_index()

In [28]:
df_av = df.groupby(['year']).mean() # find annual av for each variable

In [29]:
df_av['maxmoc'].mean()

17.064522702870917

In [30]:
df_av['maxmoc'].std()

1.4844900348651107

In [31]:
df_av['Q_sum_PW'] = df_av['Q_sum']/1E15 # MHT at 26.5degN

In [32]:
df_av['Q_sum_PW'].mean()

1.2031448616366116

In [33]:
df_av['Q_sum_PW'].std()

0.10498461049583255

### Prepping EN4 temperature and salinity data

Using Argo period (2000 onwards): using 2001 - 2020, 20 yr time period

In [162]:
#calculate annual av:
#ncra -F -d time,1,,1 EN.4.2.1.f.analysis.g10.2020*.nc EN4_2020_anav.nc

mdir = '/data/users/smoreton/Obs/EN4_analysis/raw_zipped/' 

## Mean SST over Newfoundland 

In [163]:
def find_nearest(array, value):
    n = [abs(i-value) for i in array]
    idx = n.index(min(n))
    return idx #array[idx]

In [173]:
no_years = 20 
all_years = np.linspace(2001, 2020, no_years).astype(int)
mean_SST_Newfound_allyrs = np.zeros(no_years)

for i in range(len(all_years)):
    year = all_years[i]
    
    file = 'EN4_'+str(year)+'_anav.nc'
    ds = xr.open_dataset(mdir + file)

    # converting temp from K to C
    ds['temperature'] = ds.temperature - 273.15
    ds.temperature.attrs['units'] = 'degC'

    #average SST off Newfoundland -43, -37 degW 45-50 degN
    imin = find_nearest(ds.lon, 317)
    imax = find_nearest(ds.lon, 323)
    jmin = find_nearest(ds.lat, 45)
    jmax = find_nearest(ds.lat, 50)

    mean_SST_Newfound_allyrs[i] = ds.temperature[0,0,jmin:jmax, imin:imax].mean().values #array

In [174]:
np.nanmean(mean_SST_Newfound_allyrs)

13.90855617900697

In [175]:
np.nanstd(mean_SST_Newfound_allyrs)

0.5102101109813537

## Mean SSS in Labrador Sea

In [177]:
no_years = 20 
all_years = np.linspace(2001, 2020, no_years).astype(int)
mean_SSS_allyrs = np.zeros(no_years)

for i in range(len(all_years)):
    year = all_years[i]
    
    file = 'EN4_'+str(year)+'_anav.nc'
    ds = xr.open_dataset(mdir + file)

    #average SSS: -60, -50 degW 55-62 degN 
    imin = find_nearest(ds.lon, 300)
    imax = find_nearest(ds.lon, 310)
    jmin = find_nearest(ds.lat, 55)
    jmax = find_nearest(ds.lat, 62)

    mean_SSS_allyrs[i] = ds.salinity[0,0,jmin:jmax, imin:imax].mean().values #array

In [178]:
np.nanmean(mean_SSS_allyrs)

33.97744108143307

In [179]:
np.nanstd(mean_SSS_allyrs)

0.13480149124388918

## GS separation latitude

In [33]:
def find_nearest(array, value):
    n = [abs(i-value) for i in array]
    idx = n.index(min(n))
    return idx, array[idx]

In [40]:
mdir = '/data/users/smoreton/Obs/EN4_analysis/raw_zipped/' 

# longitude & depth slice
chosen_lon = 360-72
chosen_depth = 200
isotherm = 15

In [41]:
no_years = 20 
all_years = np.linspace(2001, 2020, no_years).astype(int)

GS_sep_lat_allyrs = np.zeros(no_years)

for i in range(len(all_years)):
    year = all_years[i]
    #print (year)
    
    file = 'EN4_'+str(year)+'_anav.nc'
    ds = xr.open_dataset(mdir + file)

    # converting temp from K to C
    ds['temperature'] = ds.temperature - 273.15
    ds.temperature.attrs['units'] = 'degC'

    # selecting chosen depth
    T_z_slice = ds.sel(depth=[chosen_depth], method='nearest').temperature[0][0]

    # selecting chosen longitude
    idx_x, lon_check = find_nearest(ds.lon.values, chosen_lon) 

    # finding lat at chosen isotherm
    idx_y, T_check = find_nearest(T_z_slice.fillna(0)[:, idx_x].values, isotherm)

    GS_sep_lat_allyrs[i] = ds.lat[idx_y].values

In [42]:
GS_sep_lat_allyrs.mean()

37.2

In [43]:
GS_sep_lat_allyrs.std()

0.4

In [44]:
GS_sep_lat_allyrs

array([38., 37., 37., 37., 37., 37., 37., 37., 37., 37., 37., 37., 37.,
       37., 37., 38., 37., 37., 38., 38.])

### NAC latitude

In [58]:
# longitude & depth slice
chosen_lon = 360-41
chosen_depth = 50
isotherm = 10

no_years = 20 
all_years = np.linspace(2001, 2020, no_years).astype(int)

NAC_lat_allyrs = np.zeros(no_years)

for i in range(len(all_years)):
    year = all_years[i]
    #print (year)
    
    file = 'EN4_'+str(year)+'_anav.nc'
    ds = xr.open_dataset(mdir + file)

    # converting temp from K to C
    ds['temperature'] = ds.temperature - 273.15
    ds.temperature.attrs['units'] = 'degC'

    #cropping to NA: -80-0 degW 35-60 degN 
    imin = find_nearest(ds.lon, 280)
    imax = find_nearest(ds.lon, 360)
    jmin = find_nearest(ds.lat, 35)
    jmax = find_nearest(ds.lat, 60)
    
    # jmin[0] # 118
    # jmax[0] # 143
    # imin[0] # 279
    # imax[0] # 359
    
    ds = ds.isel(lat=slice(118,143), lon=slice(279,359))

    # selecting chosen depth
    T_z_slice = ds.sel(depth=[chosen_depth], method='nearest').temperature[0][0]

    # selecting chosen longitude
    idx_x, lon_check = find_nearest(ds.lon.values, chosen_lon) 
    
    # finding lat at chosen isotherm
    idx_y, T_check = find_nearest(T_z_slice.fillna(0)[:, idx_x].values, isotherm)
    
    #print (T_z_slice)

    NAC_lat_allyrs[i] = ds.lat[idx_y].values
    
    

In [59]:
NAC_lat_allyrs

array([51., 51., 51., 52., 52., 52., 52., 51., 51., 52., 51., 51., 51.,
       50., 50., 51., 52., 51., 51., 51.])

In [60]:
NAC_lat_allyrs.mean()

51.2

In [61]:
NAC_lat_allyrs.std()

0.6

## MXL in Labrador Sea in March

http://www.ifremer.fr/cerweb/deboyer/mld/Surface_Mixed_Layer_Depth.php

MLD in density with a fixed threshold criterion (0.03kg/m3) from 1941-2002

In [3]:
ds = xr.open_dataset('/data/users/smoreton/Obs/ifremer_mxl/mld_DR003_c1m_reg2.0.nc')

/opt/scitools/environments/default/2021_03_18-1/lib/python3.6/site-packages/xarray/coding/times.py:517: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/opt/scitools/environments/default/2021_03_18-1/lib/python3.6/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


In [9]:
#average MXL: 300-310 degW 55-62 degN 
imin = find_nearest(ds.lon, 300)
imax = find_nearest(ds.lon, 310)
jmin = find_nearest(ds.lat, 55)
jmax = find_nearest(ds.lat, 62)

In [17]:
mld_lab = ds.mld[2,jmin:jmax,imin:imax] #selecting March (3rd month)

In [23]:
mld_lab_1D = mld_lab.values.ravel()
mld_lab_1D[mld_lab_1D == 1E9] = np.nan

In [24]:
mld_lab.mean()

<xarray.DataArray 'mld' ()>
array(489.09293, dtype=float32)
Coordinates:
    time     object 0001-03-15 12:00:00

In [26]:
mld_lab.max()

<xarray.DataArray 'mld' ()>
array(974.16174, dtype=float32)
Coordinates:
    time     object 0001-03-15 12:00:00

In [27]:
mld_lab.std()

<xarray.DataArray 'mld' ()>
array(235.12221, dtype=float32)
Coordinates:
    time     object 0001-03-15 12:00:00

## Heat content of NA subpolar gyre (all depths)


sum ( rho cp T  * e1t * e2t * e3t * tmask )
https://gallery.pangeo.io/repos/NCAR/notebook-gallery/notebooks/Run-Anywhere/Ocean-Heat-Content/OHC_tutorial.html#:~:text=%20How%20to%20calculate%20ocean%20heat%20content%20%C2%B6,is%20complicated.%20But%20in%20Python%20we...%20More%20

In [203]:
#48 to 72 degrees N, size of longitude degree in m
dx = np.array([74625.33,
73171.77,
71695.73,
70197.65,
68677.99,
67137.20,
65575.75,
63994.10,
62392.75,
60772.16,
59132.84,
57475.28,
55799.98,
54107.46,
52398.22,
50672.80,
48931.72,
47175.51,
45404.71,
43619.86,
41821.51,
40010.21,
38186.53,
36351.01]
)
dx.mean()

56222.00291666666

In [208]:
def calc_ocean_heat(dz, temperature):
    #sum(e1t * e2t * e3t * temp) * rho * cp 
    
    #from cdftools:
    rho = 1020 #kg/m^3
    c_p = 4000 #J/(kg K)
    
    # weight by dz
    weighted_temperature = temperature * dz[:, None, None] 
    #temperature is 3D array
    
    heat = weighted_temperature.sum(dim="depth")*rho*c_p
    heat = heat.sum(dim="lat")
    heat = heat.sum(dim="lon")
    
    # creating approximate e1t and e2t grid for obs: 1deg (~111km in lat, 
    #using the mean lon degree size in m between 48-72degN)  
    heat = ((40*56222) + (24*111000)) * heat #degrees to m
    
    return heat

#if wanted to filter depth:
#ds.depth.where(ds.depth < 50, drop = True)

In [209]:
no_years = 20 
all_years = np.linspace(2001, 2020, no_years).astype(int)

# no_years = 1
# all_years = np.linspace(2001, 2001, no_years).astype(int)

gyre_heat_content_allyrs = np.zeros(no_years)

for i in range(len(all_years)):
    year = all_years[i]
    
    file = 'EN4_'+str(year)+'_anav.nc'
    ds = xr.open_dataset(mdir + file)

    # converting temp from K to C
    ds['temperature'] = ds.temperature - 273.15
    ds.temperature.attrs['units'] = 'degC'

    #heat content (J) in NA: 300-340 degW 48-72 degN
    imin = find_nearest(ds.lon, 300)
    imax = find_nearest(ds.lon, 340)
    jmin = find_nearest(ds.lat, 48)
    jmax = find_nearest(ds.lat, 72)

    temp_reg_3D = ds.temperature[0,0:-1,jmin:jmax, imin:imax] #array
    
    dz = (np.diff(ds.depth.values)) *1E3 #km to m
    
    gyre_heat_content_allyrs[i] = calc_ocean_heat(dz, temp_reg_3D)

In [182]:
#total depth in m:
dz.sum()

5345250.0

In [210]:
gyre_heat_content_allyrs

array([1.51272705e+23, 1.51850618e+23, 1.52456684e+23, 1.55319075e+23,
       1.55628934e+23, 1.55972266e+23, 1.55846250e+23, 1.54002259e+23,
       1.53703574e+23, 1.56457821e+23, 1.57108204e+23, 1.55169851e+23,
       1.56565721e+23, 1.53151851e+23, 1.50380226e+23, 1.50066462e+23,
       1.52344263e+23, 1.50977438e+23, 1.52159788e+23, 1.50649511e+23])

In [211]:
gyre_heat_content_allyrs.mean()

1.5355417513870916e+23

In [212]:
gyre_heat_content_allyrs.std()

2.2547946295631185e+21

## Overflow temperature & salinity

In [3]:
isopyc_thres = 27.8

## OVIDE

Source : https://www.seanoe.org/data/00353/46446/

2002-2012, in 2 yr intervals e.g. 2002, 2004, etc. so 6 yrs in total

In [6]:
ds = xr.open_dataset('/data/users/smoreton/Obs/NA_overflows/ovide/gridded_data_along_OVIDE_line.nc')
ds.Year.values

array([2002., 2004., 2006., 2008., 2010., 2012.])

In [8]:
# cropped to Icelandic basin only
ds = ds.isel(Dist=slice(99, 214))
T_27_8_obs_each_year = ds.TPOT.where(ds.SIG0.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='Depth').mean(skipna=True, keep_attrs=True, dim='Dist')

S_27_8_obs_each_year = ds.SAL.where(ds.SIG0.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='Depth').mean(skipna=True, keep_attrs=True, dim='Dist')

In [9]:
S_27_8_obs_each_year.mean().values

array(34.94983169)

In [10]:
S_27_8_obs_each_year.std().values

array(0.00509406)

In [11]:
T_27_8_obs_each_year.mean().values

array(2.90064253)

In [12]:
T_27_8_obs_each_year.std().values

array(0.04285094)

### Overflow T and S using nordic-seas-validation output

In [49]:
main_dir = '/data/users/smoreton/Obs/NA_overflows/'

ovide_ds = xr.open_dataset(main_dir+'ovide_Xsection.nc') #no time variable so using my original dataset
denmarkstr_ds = xr.open_dataset(main_dir+'latrabjarg_clim_Xsection.nc')  #no time variable so no std
eel_ds = xr.open_dataset(main_dir+'eel_Xsection.nc') 
kogur_ds = xr.open_dataset(main_dir+'kogur_Xsection.nc') 


In [47]:
#denmarkstr_ds, 
T_27_8_obs_each_year = denmarkstr_ds.potential_temperature.where(denmarkstr_ds.sigma_theta.values > isopyc_thres).mean(skipna=True, keep_attrs=True)

S_27_8_obs_each_year = denmarkstr_ds.practical_salinity.where(denmarkstr_ds.sigma_theta.values > isopyc_thres).mean(skipna=True, keep_attrs=True)

In [50]:
T_27_8_obs_each_year.values #assume mean, no time variable

array(0.69057989)

In [51]:
S_27_8_obs_each_year.values

array(34.84691209)

In [72]:
#eel_ds, changes each year
T_27_8_obs_each_year = eel_ds.potential_temperature.where(eel_ds.sigma_theta.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

S_27_8_obs_each_year = eel_ds.practical_salinity.where(eel_ds.sigma_theta.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

In [74]:
S_27_8_obs_each_year.mean().values

array(34.95592763)

In [75]:
S_27_8_obs_each_year.std().values

array(0.01379818)

In [76]:
T_27_8_obs_each_year.mean().values

array(3.21952773)

In [77]:
T_27_8_obs_each_year.std().values

array(0.11272316)

In [69]:
eel_ds.time.values #1996-2017, 15 years

array(['1996-01-01T00:00:00.000000000', '1997-01-01T00:00:00.000000000',
       '1998-01-01T00:00:00.000000000', '1999-01-01T00:00:00.000000000',
       '2000-01-01T00:00:00.000000000', '2005-01-01T00:00:00.000000000',
       '2006-01-01T00:00:00.000000000', '2009-01-01T00:00:00.000000000',
       '2010-01-01T00:00:00.000000000', '2011-01-01T00:00:00.000000000',
       '2013-01-01T00:00:00.000000000', '2014-01-01T00:00:00.000000000',
       '2015-01-01T00:00:00.000000000', '2016-01-01T00:00:00.000000000',
       '2017-01-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [80]:
#kogur_ds, changes each year
T_27_8_obs_each_year = kogur_ds.potential_temperature.where(kogur_ds.sigma_theta.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

S_27_8_obs_each_year = kogur_ds.salinity.where(kogur_ds.sigma_theta.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

In [81]:
S_27_8_obs_each_year.mean().values

array(34.88384838)

In [82]:
S_27_8_obs_each_year.std().values

array(0.00559513)

In [88]:
T_27_8_obs_each_year.mean().values

array(0.35019253)

In [89]:
T_27_8_obs_each_year.std().values

array(0.08281083)

In [90]:
kogur_ds.time.values #2011 sept - 2012 july

array(['2011-09-01T00:00:00.000000000', '2011-09-01T08:00:00.000000000',
       '2011-09-01T16:00:00.000000000', ...,
       '2012-07-30T16:00:00.000000000', '2012-07-31T00:00:00.000000000',
       '2012-07-31T08:00:00.000000000'], dtype='datetime64[ns]')

In [44]:
#osnap_ds, changes each year
# I calculated density & saved within model file, so accessing it there 
main_dir = '/data/users/smoreton/VALNA/DATA/u-ak108/'
osnap_ds = xr.open_dataset(main_dir+'nemo_ak108o_1y_19501201_osnap_Xsection.nc') 

osnap_ds = osnap_ds.isel(station=slice(76, 205)) # Irminger and Icelandic basins,  

T_27_8_obs_each_year = osnap_ds.obs_temperature.where(osnap_ds.obs_density.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

S_27_8_obs_each_year = osnap_ds.obs_salinity.where(osnap_ds.obs_density.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')


In [45]:
T_27_8_obs_each_year.mean().values

array(2.96945723)

In [46]:
T_27_8_obs_each_year.std().values

array(0.01964481)

In [47]:
S_27_8_obs_each_year.mean().values

array(34.947037, dtype=float32)

In [48]:
S_27_8_obs_each_year.std().values

array(0.00142154, dtype=float32)

In [19]:
osnap_ds.time.values #2014-2016

array(['2014-07-31T00:00:00.000000000', '2014-08-30T00:00:00.000000000',
       '2014-09-29T00:00:00.000000000', '2014-10-29T00:00:00.000000000',
       '2014-11-28T00:00:00.000000000', '2014-12-28T00:00:00.000000000',
       '2015-01-27T00:00:00.000000000', '2015-02-26T00:00:00.000000000',
       '2015-03-28T00:00:00.000000000', '2015-04-27T00:00:00.000000000',
       '2015-05-27T00:00:00.000000000', '2015-06-26T00:00:00.000000000',
       '2015-07-26T00:00:00.000000000', '2015-08-25T00:00:00.000000000',
       '2015-09-24T00:00:00.000000000', '2015-10-24T00:00:00.000000000',
       '2015-11-23T00:00:00.000000000', '2015-12-23T00:00:00.000000000',
       '2016-01-22T00:00:00.000000000', '2016-02-21T00:00:00.000000000',
       '2016-03-22T00:00:00.000000000'], dtype='datetime64[ns]')

In [53]:
main_dir = '/data/users/smoreton/VALNA/DATA/u-ak108/'
osnap_ds = xr.open_dataset(main_dir+'nemo_ak108o_1y_19501201_osnap_Xsection.nc') 

osnap_ds = osnap_ds.isel(station=slice(76, 143)) # Irminger basin only

T_27_8_obs_each_year = osnap_ds.obs_temperature.where(osnap_ds.obs_density.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

S_27_8_obs_each_year = osnap_ds.obs_salinity.where(osnap_ds.obs_density.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')


In [54]:
T_27_8_obs_each_year.mean().values

array(2.90767408)

In [55]:
T_27_8_obs_each_year.std().values

array(0.0308461)

In [56]:
S_27_8_obs_each_year.mean().values

array(34.933804, dtype=float32)

In [57]:
S_27_8_obs_each_year.std().values

array(0.0020512, dtype=float32)

In [59]:
main_dir = '/data/users/smoreton/VALNA/DATA/u-ak108/'
osnap_ds = xr.open_dataset(main_dir+'nemo_ak108o_1y_19501201_osnap_Xsection.nc') 

osnap_ds = osnap_ds.isel(station=slice(143, 205)) #Icelandic basin only

T_27_8_obs_each_year = osnap_ds.obs_temperature.where(osnap_ds.obs_density.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

# S_27_8_obs_each_year = osnap_ds.obs_salinity.where(osnap_ds.obs_density.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')


In [60]:
T_27_8_obs_each_year.mean().values

array(3.03233039)

In [61]:
T_27_8_obs_each_year.std().values

array(0.0182862)

In [62]:
S_27_8_obs_each_year.mean().values

array(34.933804, dtype=float32)

In [63]:
S_27_8_obs_each_year.std().values

array(0.0020512, dtype=float32)

In [69]:
#hansen_ds, no time dim
# I calculated density & saved within model file, so accessing it there 
main_dir = '/data/users/smoreton/VALNA/DATA/u-ar435/'
hansen_ds = xr.open_dataset(main_dir+'nemo_ar435o_1y_20041201_hansen_Xsection.nc') 

T_27_8_obs = hansen_ds.obs_temperature.where(hansen_ds.obs_density.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

S_27_8_obs = hansen_ds.obs_salinity.where(hansen_ds.obs_density.values > isopyc_thres).mean(skipna=True, keep_attrs=True, dim='depth').mean(skipna=True, keep_attrs=True, dim='station')

In [71]:
T_27_8_obs.values

array(0.05831493)

In [72]:
S_27_8_obs.values

array(34.90626667)